<a href="https://colab.research.google.com/github/InowaR/colab/blob/main/ICP_scans_simple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [306]:
import numpy as np
from scipy.spatial import KDTree

NUM_POINTS = 40
NOISE_LEVEL = 0.05
ITERS = 30

def icp(src, tgt):
    tree = KDTree(tgt)
    R, t = np.eye(2), np.zeros(2)
    for _ in range(ITERS):
        tr = src @ R.T + t
        idx = tree.query(tr)[1]
        src_m, tgt_m = tr.mean(0), tgt[idx].mean(0)
        H = (tr - src_m).T @ (tgt[idx] - tgt_m)
        U, _, Vt = np.linalg.svd(H)
        R_new = Vt.T @ U.T
        t_new = tgt_m - R_new @ src_m
        R, t = R_new @ R, R_new @ t + t_new
    return [t[0], t[1], np.arctan2(R[1,0], R[0,0])]

def create_scans():
    w, h = 5, 2
    corners = [[-w,-h], [-w,h], [w,h], [w,-h]]

    points = []
    for i in range(4):
        start = corners[i]
        end = corners[(i+1)%4]
        line = np.linspace(0, 1, NUM_POINTS)[:, None] * (np.array(end)-np.array(start)) + start
        points.append(line)
    points = np.vstack(points)[:-1]

    transforms = [
        [0, 0, 0],
        [1, 0, 0],
        [0, 0, np.radians(45)],
        [1, 1, 0],
        [0, 0, 0],
        [0, 0, np.radians(-45)],
        [0, 0, np.radians(-45)],
        [1, -1, 0],
        [1, -1, 0]
    ]

    scans = []
    for dx, dy, da in transforms:
        c, s = np.cos(da), np.sin(da)
        R = [[c, -s], [s, c]]
        scan = points @ R + [dx, dy]
        scans.append(scan + np.random.randn(*scan.shape)*NOISE_LEVEL)

    return scans

def process_sequentially(scans):
    aligned = [scans[0]]
    robots = [(0, 0, 0)]

    for i in range(1, len(scans)):
        dx, dy, da = icp(aligned[-1], scans[i])
        R = [[np.cos(da), -np.sin(da)], [np.sin(da), np.cos(da)]]
        aligned_scan = (scans[i] - [dx, dy]) @ R
        aligned.append(aligned_scan)

        prev_robot = robots[-1]
        new_robot = (prev_robot[0] + dx, prev_robot[1] + dy, prev_robot[2] - da)
        robots.append(new_robot)

        print(f"x={round(new_robot[0])}, y={round(new_robot[1])}, угол={round(np.degrees(new_robot[2]))}°")

scans = create_scans()
process_sequentially(scans)

x=1, y=0, угол=0°
x=1, y=0, угол=45°
x=2, y=1, угол=45°
x=2, y=1, угол=45°
x=2, y=1, угол=0°
x=2, y=1, угол=-45°
x=3, y=0, угол=-45°
x=4, y=-1, угол=-45°
